TODO: Read data as JSON directly from https://services5.arcgis.com/ROBnTHSNjoZ2Wm1P/arcgis/rest/services/COVID_19_Statistics/FeatureServer/3/query?where=1%3D1&outFields=dtcreate,Alameda,GlobalID,OBJECTID&outSR=4326&f=json and then transform to csv

Now we have a CSV format - load it (this can be elided if we load and transform the JSON directly

TODO: Rename "I" to "Total Cases"

In [ ]:
import os

#LOCAL_DATA_ROOT = "/Users/jason/development/cv19/"
LOCAL_DATA_ROOT = "./"
CSV_FILE = "alameda_data.csv"
CSV_OUTPUT_FILE = "alameda_delta_data.csv"

CASES_PER_DAY_GRAPH_FILE = "alameda_daily_cases.png"

CSV_PATH = os.path.join(LOCAL_DATA_ROOT, CSV_FILE)
CSV_OUTPUT_PATH = os.path.join(LOCAL_DATA_ROOT, CSV_OUTPUT_FILE)
CASES_PER_DAY_GRAPH_PATH = os.path.join(LOCAL_DATA_ROOT, CASES_PER_DAY_GRAPH_FILE)

%matplotlib auto
import pandas as pd

def load_covid_data(csv_path=CSV_PATH):
    return pd.read_csv(csv_path)

In [ ]:
covid = load_covid_data()
covid.head()

In [ ]:
covid.info()

Clean up the data - replace NaN with 0; replace <10 with 0

In [ ]:
covid["Total"].fillna(0,inplace=True)
covid["Total"].replace("<10", 0,inplace=True)

Convert I field from an object (as it originally had strings) to an int

In [ ]:
covid["Total"] = covid["Total"].astype(str).astype(int)

In [ ]:
covid.info()


In [ ]:
covid.head()

In [ ]:
pd.set_option('display.max_rows', None)
print(covid)

Work out the case delta between each row to generate a number of new cases per day as the second column

In [ ]:
test = covid["Total"]
print(test)


UNKNOWN: Why is diff on a specific row (or on test) not working?
The "I" column is an object, not an int - convert?

In [ ]:
test.diff()

Convert dates into standard pandas data format

In [ ]:
covid['std-date'] = pd.to_datetime(covid["dates"], format='%d/%m/%Y')
print(covid)
covid.info()

Need to sort by incrementing dates

In [ ]:
covid = covid.sort_values(by='std-date',ascending=True) 

In [ ]:
print(covid)

Work out diffs; convert NaN to the original value and convert diff column to int
TODO: Move diff column to column two

In [ ]:
covid["I"] = covid["Total"].diff().fillna(covid['Total']).astype(int)

In [ ]:
print(covid)

In [ ]:
covid.info()

Swap cells to be in the correct organization for WHO-PAHO

In [ ]:
cols = list(covid.columns)
a, b = cols.index('Total'), cols.index('I')
cols[b], cols[a] = cols[a], cols[b]
covid = covid[cols]


In [ ]:
print(covid)

We have a lot of dates with no data (at 0 total value) at the end of the dataset; drop these invalid data. 

TODO: We currently do it from a known end point, but we should look for where I goes negative (or we transition to 0) and drop all after this point

In [ ]:
#covid = covid.loc[0:276]
#print(covid)

Remove the std-date column from the dataframe we write as CSV as it confused WHO-PAHO

In [ ]:
csv_covid = covid.drop(['std-date'], axis=1)

In [ ]:
csv_covid.to_csv(CSV_OUTPUT_PATH, header=True, index=False) 

Display Alameda covid cases over time

In [ ]:
covid.plot(x="std-date", y="Total", title="Alameda Covid cumulative cases")

Show log y - change in growth over time

In [ ]:
covid.plot(x="std-date", y="Total",logy=True)

In [ ]:
ax = covid.plot(x="std-date", y="I", kind="bar", figsize=(20,10), title="Alameda Covid cases per day")

n = 10

ticks = ax.xaxis.get_ticklocs()
ticklabels = [l.get_text() for l in ax.xaxis.get_ticklabels()]
ax.xaxis.set_ticks(ticks[::n])
ax.xaxis.set_ticklabels(ticklabels[::n])
ax.figure.tight_layout()

ax.figure.show()


In [ ]:
ax.figure.savefig(CASES_PER_DAY_GRAPH_PATH)